In [1]:
import sys
sys.path.append('//anaconda/lib/python2.7/site-packages/')
from IPython.display import display
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold
from sklearn.cluster import FeatureAgglomeration
from sklearn import svm
import matplotlib
import matplotlib.pyplot as plt
import nltk
from datetime import datetime as dtdm
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, euclidean
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from textblob import TextBlob
from datetime import datetime as dt
import math
from geopy.distance import vincenty
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

//anaconda/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Attractions

This section of our report is based off of the principle tenent of real estate: location, location, location.  We scraped the top tourist attractions in New York and made indicator variables to see if listings were nearby.  Ultimately, we wanted to know whether or not proximity to these destinations affected price in any way.  After running this code, a new dataset CSV was created.

In [3]:
#read in the dataset
attractions = pd.read_csv('attractions_data.csv')

In [25]:
attractions

,attraction,lat,lon
0,Central Park,40.782865,-73.965355
1,American Museum of Natural History,40.781324,-73.973988
2,Rockefeller Center and Top of the Rock Observa...,40.759245,-73.979304
3,St. Patrick's Cathedral,40.758465,-73.975993
4,Bryant Park,40.753597,-73.983233
5,Metropolitan Museum of Art,40.779437,-73.963244
6,National September 11 Memorial & Museum,40.710580,-74.015583
7,Grand Central Station,40.750936,-73.977105
8,Empire State Building,40.748440,-73.985655
9,Statue of Liberty,40.689249,-74.044500


In [26]:
final_df_small = final_df[:100]

In [4]:
#function to count the number of nearby attractions
def attract_count(df):
    a_sum = 0
    for i,attract in attractions.iterrows():
        v1 = (df.latitude,df.longitude)
        v2 = (attract.lat,attract.lon)
        dist = vincenty(v1, v2).meters
        if dist < 1000:
            a_sum += 1
    return a_sum

In [5]:
#add a column of attraction counts
final_df['attraction_count'] = final_df.apply(attract_count,axis=1)

In [7]:
final_df[:100]

,Unnamed: 0,Allerton,Alphabet City,Astoria,Bath Beach,Battery Park City,Bay Ridge,Baychester,Bayside,Bedford Park,...,host_listing_count,host_since,first_review,last_review,sentiment,New Years,July 4th,Christmas,New Years Eve,attraction_count
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,261014400.0,262569600.0,314236800.0,0.283516,0,0,0,0,1
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,252806400.0,286934400.0,315273600.0,0.437795,0,0,1,1,0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,176947200.0,181958400.0,310089600.0,0.308604,0,0,0,0,0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,308793600.0,313459200.0,313459200.0,0.000000,0,1,1,0,1
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,190166400.0,206496000.0,313718400.0,0.122600,0,0,0,0,0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,251078400.0,295660800.0,314668800.0,0.454705,0,0,1,0,0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,255484800.0,296265600.0,305078400.0,0.518079,0,0,0,0,0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25,202348800.0,213667200.0,310521600.0,0.700000,0,0,0,0,7
8,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,280972800.0,284169600.0,311558400.0,0.381082,0,1,1,0,0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,268704000.0,292464000.0,314236800.0,0.114130,0,0,0,0,7


In [ ]:
#print the attraction counts to a csv
final_df.to_csv('attractions_added.csv')

In [7]:
#drop the first row
final_df = final_df.drop('Unnamed: 0',axis=1)

In [3]:
#read in the prices
prices = pd.read_csv('reduced_prices.csv')

In [4]:
#clean up the prices data
prices = prices.drop('Unnamed: 0',axis=1)
prices.reset_index(inplace=True)
prices=prices.drop('index',axis=1)

In [5]:
from sklearn.ensemble import RandomForestRegressor

In [8]:
#split the data
x_train,x_test,y_train,y_test = train_test_split(final_df,prices,test_size=0.3)

In [9]:
x_train.shape

(12612, 225)

In [29]:
y_train[:5]

,price
17646,140
16502,325
2816,110
13353,295
11487,160


In [10]:
forest = RandomForestRegressor()
forest.fit(x_train,y_train.values)
forest.score(x_test,y_test.values)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


0.39615686934477085

In [25]:
display(prices[:5].T)

,0,1,2,3,4
price,160,58,185,195,165


In [7]:
final_df = pd.read_csv('attractions_added.csv')

In [ ]:
prices.to_csv('.csv')